# Lab Assignment 4: Using APIs in Python
## DS 6001: Practice and Application of Data Science

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

In this lab, you will work with the public API provided by [genius.com](https://genius.com/), a website that calls itself "the world’s biggest collection of song lyrics and musical knowledge." You will need to read the API documentation carefully, acquire an access key, and use it without sharing it to pull data from this API into Python. You will also practice using a library made specifically to wrap around `requests` to make calling from the Genius API easier.

## Problem 0
Import the following libraries:

In [2]:
import numpy as np
import pandas as pd
import requests
import json
import os
import dotenv
import sys
sys.tracebacklimit = 0 # turn off the error tracebacks

## Problem 1
The Genius API documentation is here: https://docs.genius.com/#/getting-started-h1. Read through the documentation carefully. Although the Genius API is free and public, it still requires users an access key to use the API. In this case, Genius provides users with three codes: a client ID, a client ID secret, and a client access token. Use the documentation to find a way to obtain these codes for yourself. Write a paragraph that describes all of the steps you needed to take (but DO NOT list your access codes in this paragraph).

Some hints and cautions: 

1. Before you can use the API, you will need a regular, free user account with Genius. Sign-up here: https://genius.com/signup_or_login

2. Genius's API is built to support third-party app development, not data scientists. The language is entirely geared toward app development. Under "Authentication" there are instructions to third-party developers for guiding their own users in getting access keys. That's not relevant to getting access for yourself. It's not hard to get an access key, but the guidance here is not very clear. Be patient and read everything in the Authentication section carefully.

3. When you arrive at the page that allows you to register for API access keys, the language is still geared toward app development. You will be prompted to name your app and provide the URLs associated with the app. It doesn't much matter what you name your app, and I just used the Collab main page (https://collab.its.virginia.edu/portal?containerLogin=true) for the URLs.

4. When you get your codes, copy them in a text file. In problem 2 you will copy these codes over again to a `.env` file. [4 points]

I created my genius account. Then, I went to developers on the bottom of the Genius page. I clicked on "Getting Started" and made my API client. I named my app "DS6001" and used the URL "https://collab.its.virginia.edu/portal?containerLogin=true". I copied the client ID, client secret, and client access token to a text file.

## Problem 2
Create a `.env` file for this project. Open it, copy your access codes into it, and save it. Then use Python code to load the environmental variables contained in the `.env` file, and create variables that contain each of the three codes. (You can print these variables to make sure it worked, but do not allow your access keys to display in your notebook). [4 points]

In [3]:
dotenv.load_dotenv()
geniuskey = os.getenv('geniuskey')
genius_client_id = os.getenv('genius_client_id')
genius_client_access_token = os.getenv('genius_client_access_token')
redirect_uri = os.getenv('redirect_uri')


## Problem 3
The root for all Genius APIs is https://api.genius.com. Find the endpoint for the Search API. (You will have to click the "Authorize with Genius" button in the upper-right corner if you haven't already done so). Use the `requests` library to issue a search for Bob Dylan. Genius's API is organized in a way that every individual artist has his or her own API endpoint. Display a portion of the JSON output that displays the API endpoint path for the data on Bob Dylan. 

Hint: to authenticate, specify your access token (not your client ID or client secret) as the `access_token` parameter. You will have to dig around the JSON output to find the artist ID, but it is listed under `primary_artist` several branches down the JSON tree. [4 points]

In [4]:
url = 'https://httpbin.org/user-agent'
r = requests.get(url)
my_json = json.loads(r.text)
useragent = my_json['user-agent']
headers = {'User-Agent': useragent,
           'From': 'ljn5yms@virginia.edu'}

In [5]:
root = 'https://api.genius.com'
endpoint = '/search'
params = {"q": "Bob Dylan",
          "access_token": genius_client_access_token}
r = requests.get(root + endpoint,params=params,headers=headers)
r

<Response [200]>

In [6]:
my_json = json.loads(r.text)
my_json

{'meta': {'status': 200},
 'response': {'hits': [{'highlights': [],
    'index': 'song',
    'type': 'song',
    'result': {'annotation_count': 12,
     'api_path': '/songs/79424',
     'artist_names': 'Bob Dylan',
     'full_title': "Blowin' in the Wind by\xa0Bob\xa0Dylan",
     'header_image_thumbnail_url': 'https://images.genius.com/84e1705bc644951972168d1015fa9024.300x300x1.jpg',
     'header_image_url': 'https://images.genius.com/84e1705bc644951972168d1015fa9024.1000x1000x1.jpg',
     'id': 79424,
     'lyrics_owner_id': 73267,
     'lyrics_state': 'complete',
     'path': '/Bob-dylan-blowin-in-the-wind-lyrics',
     'primary_artist_names': 'Bob Dylan',
     'pyongs_count': 48,
     'relationships_index_url': 'https://genius.com/Bob-dylan-blowin-in-the-wind-sample',
     'release_date_components': {'year': 1963, 'month': 5, 'day': 27},
     'release_date_for_display': 'May 27, 1963',
     'release_date_with_abbreviated_month_for_display': 'May. 27, 1963',
     'song_art_image_thum

In [7]:
my_json['response']['hits'][0]['result']['primary_artist']

{'api_path': '/artists/181',
 'header_image_url': 'https://images.genius.com/e87fb11dd7f33cc7fd1a8915ad36f7c9.1000x667x1.png',
 'id': 181,
 'image_url': 'https://images.genius.com/571780cc8959528e8166e5894ab13d6f.1000x1000x1.jpg',
 'is_meme_verified': False,
 'is_verified': False,
 'name': 'Bob Dylan',
 'url': 'https://genius.com/artists/Bob-dylan'}

## Problem 4
Add `/songs` to the end of the the endpoint path you found in problem 3 and use this path to request the 20 most popular Bob Dylan songs. Organize these data in a `pandas` data frame. [4 points]

In [8]:
root = 'https://api.genius.com'
endpoint = '/artists/181/songs/'
params = {"access_token": genius_client_access_token}
r = requests.get(root + endpoint,params=params,headers=headers)
r

<Response [200]>

In [9]:
my_json = json.loads(r.text)
my_json

{'meta': {'status': 200},
 'response': {'songs': [{'annotation_count': 0,
    'api_path': '/songs/200681',
    'artist_names': 'Bob Dylan',
    'full_title': '10,000 Men by\xa0Bob\xa0Dylan',
    'header_image_thumbnail_url': 'https://images.genius.com/9a0da499b7383ddf232734cfb860d33b.300x301x1.jpg',
    'header_image_url': 'https://images.genius.com/9a0da499b7383ddf232734cfb860d33b.500x501x1.jpg',
    'id': 200681,
    'lyrics_owner_id': 225680,
    'lyrics_state': 'complete',
    'path': '/Bob-dylan-10000-men-lyrics',
    'primary_artist_names': 'Bob Dylan',
    'pyongs_count': None,
    'relationships_index_url': 'https://genius.com/Bob-dylan-10000-men-sample',
    'release_date_components': {'year': 1990, 'month': 9, 'day': 10},
    'release_date_for_display': 'September 10, 1990',
    'release_date_with_abbreviated_month_for_display': 'Sep. 10, 1990',
    'song_art_image_thumbnail_url': 'https://images.genius.com/9a0da499b7383ddf232734cfb860d33b.300x301x1.jpg',
    'song_art_image_

In [24]:
bob_dylan_songs = pd.json_normalize(json.loads(r.text),record_path=['response','songs'])
top20 = bob_dylan_songs['full_title'][0:20]
top20


0                               10,000 Men by Bob Dylan
1        2 Dollars and 99 Cents by Bob Dylan & The Band
2                                    2 X 2 by Bob Dylan
3                              32-20 Blues by Bob Dylan
4                          4th Time Around by Bob Dylan
5                    4th Time Around by Robyn Hitchcock
6                   7 Deadly Sins by Traveling Wilburys
7                   900 Miles from My Home by Bob Dylan
8        900 Miles from My Home by Bob Dylan & The Band
9     900 Miles From My Home / Confidential (Hidden ...
10                                       A by Bob Dylan
11               Abandoned Love by ​The Everly Brothers
12    Abandoned Love by Bob Dylan (Ft. George Harrison)
13                      Abandoned Love by Gillian Welch
14    Abandoned Love by Paul Rodgers (Ft. Nils Lofgren)
15    Abraham, Martin, and John (Rehearsal) by Bob D...
16                  Absolutely Sweet Marie by Bob Dylan
17            Absolutely Sweet Marie by George H

## Problem 5
Install and import the `lyricsgenius` library in Python, which is a wrapper around `requests` that works specifically with the Genius API. . Follow the guide on the GitHub repository for this library (https://github.com/johnwmillr/LyricsGenius) for instructions on using the library. Use the `lyricsgenius` library to download and display the lyrics to "Tangled Up in Blue" by Bob Dylan. [4 points]

In [31]:
import lyricsgenius
genius = lyricsgenius.Genius(genius_client_access_token)

In [32]:
artist = genius.search_artist("Bob Dylan", max_songs=3, sort="title", include_features=True)



Searching for songs by Bob Dylan...

Song 1: "10,000 Men"
Song 2: "2 X 2"
Song 3: "32-20 Blues"

Reached user-specified song limit (3).
Done. Found 3 songs.


In [33]:
song = artist.song("Tangled Up in Blue")
print(song.lyrics)

Searching for "Tangled Up in Blue" by Bob Dylan...
Done.
96 ContributorsTangled Up in Blue Lyrics[Verse 1]
Early one morning the sun was shining
I was laying in bed
Wondering if she'd changed at all
If her hair was still red
Her folks they said our lives together
Sure was going to be rough
They never did like Mama's homemade dress
Papa's bankbook wasn't big enough
And I was standing on the side of the road
Rain falling on my shoes
Heading out for the East Coast
Lord knows I've paid some dues
Getting through
Tangled up in blue

[Verse 2]
She was married when we first met
Soon to be divorced
I helped her out of a jam, I guess
But I used a little too much force
We drove that car as far as we could
Abandoned it out west
Split up on a dark sad night
Both agreeing it was best
She turned around to look at me
As I was walking away
I heard her say over my shoulder
"We'll meet again someday
On the avenue"
Tangled up in blue
See Bob Dylan LiveGet tickets as low as $24You might also like[Verse 3]
